<a href="https://colab.research.google.com/github/yasmienAshraf/Project1/blob/main/pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Load Libraries and Data**

In [ ]:
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, AdamW
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm, trange
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df_train = pd.read_csv("/content/drive/My Drive/AI Project/AraBERT_Tokenized_Labeled_Data.csv", sep=",")
# df_train.head()

In [ ]:
# df_test = pd.read_csv("/content/drive/My Drive/AI Project/All_test_AraBERT_Tokenized_Labeled_Data.csv", sep=",")
# df_test.head()

###**Convert String Representations of Lists to Actual Lists**

In [ ]:
# #Convert string representations of lists back to actual lists
# df_train['input_ids'] = df_train['input_ids'].apply(ast.literal_eval)
# df_train['attention_mask'] = df_train['attention_mask'].apply(ast.literal_eval)
# df_test['input_ids'] = df_test['input_ids'].apply(ast.literal_eval)
# df_test['attention_mask'] = df_test['attention_mask'].apply(ast.literal_eval)


###**Define the Custom Dataset Class**

In [ ]:
# Step 4: Define Dataset class
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.input_ids = dataframe['input_ids'].tolist()
        self.attention_masks = dataframe['attention_mask'].tolist()
        self.labels = dataframe['Polarity'].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


###**Create DataLoader for Training and Testing Data**

In [ ]:
# Step 5: Create the dataset and dataloader for training and testing
# train_dataset = MyDataset(df_train)
# test_dataset = MyDataset(df_test)

# train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=12, pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)

###**Load the Pretrained Model and Optimizer**

In [ ]:
# Step 6: Load the model and optimizer
model = BertForSequenceClassification.from_pretrained('aubmindlab/bert-base-arabertv02', num_labels=3)
# optimizer = AdamW(model.parameters(), lr=2e-5)
best_checkpoint = torch.load('/content/drive/My Drive/AI Project/best_model.pt')
model.load_state_dict(best_checkpoint['model_state_dict'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-2247bbfbde56>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_glob

<All keys matched successfully>

###**Evaluate the Model on the Test Set**

In [ ]:
# Step 8: Evaluation on the test set after training for 30 epochs
def test(epoch_num):
  model.eval()
  correct = 0
  total = 0
  tbar = tqdm(test_loader, desc='Testing')
  pred_list = []
  label_list = []
  total_loss = 0
  with torch.no_grad():
      for batch in tbar:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          logits = outputs.logits
          predictions = torch.argmax(logits, dim=-1)
          pred_list.extend(predictions.cpu().numpy())
          label_list.extend(labels.cpu().numpy())

          correct += (predictions == labels).sum().item()
          total += labels.size(0)


  accuracy = accuracy_score(label_list, pred_list)
  # f1_score_macro = f1_score(label_list, pred_list, average='macro')
  # recall_score_macro = recall_score(label_list, pred_list, average='macro')
  # precision_score_macro = precision_score(label_list, pred_list, average='macro')
  # print(f"Test Accuracy after {epoch_num} Epochs: {accuracy:.4f}")
  # print(f"Test F1 Score after {epoch_num} Epochs: {f1_score_macro:.4f}")
  # print(f"Test Recall Score after {epoch_num} Epochs: {recall_score_macro:.4f}")
  # print(f"Test Precision Score after {epoch_num} Epochs: {precision_score_macro:.4f}")
  # print('='*20)
  print(f'REPORT FOR {epoch_num}')
  print(classification_report(label_list, pred_list, target_names=['NEGATIVE', 'POSITIVE', 'NEUTRAL']))
  print('=='*20)
  print(accuracy)
  return accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# test(0)

###**Training Loop for 30 Epochs**

In [ ]:
# Step 7: Training loop with 30 epochs
# epochs = 30  # Number of epochs
# ebar = trange(epochs, desc='Epochs')
# best_loss = 0
# for epoch in ebar:
#     model.train()
#     total_loss = 0
#     tbar = tqdm(train_loader, desc='Training')
#     for batch in tbar:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         # Forward pass
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         tbar.set_postfix({'Loss': f'{loss.item():.4f}'})
#         # Print the loss for this mini-batch
#         #print(f"Epoch [{epoch + 1}/{epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

#     # Print the average loss for this epoch
#     avg_loss = total_loss / len(train_loader)
#     print(f"Epoch [{epoch + 1}/{epochs}] Average Loss: {avg_loss:.4f}")
#     val_loss = test(epoch+1)

#     if val_loss > best_loss:
#       torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'val_loss': val_loss,
#             }, '/content/drive/My Drive/AI Project/best_model.pt')
#       best_loss = val_loss
#       print(f'NEW BEST LOSS FOUND {best_loss:.4f}')


Testing

In [ ]:
df_sport = pd.read_csv("/content/drive/My Drive/AI Project/Sport_testing_AraBERT_Tokenized_Labeled_Data.csv", sep=",")
df_tech = pd.read_csv("/content/drive/My Drive/AI Project/Tech_test_AraBERT_Tokenized_Labeled_Data.csv", sep=",")
df_political = pd.read_csv("/content/drive/My Drive/AI Project/Political_test_AraBERT_Tokenized_Labeled_Data.csv", sep=",")
df_social = pd.read_csv("/content/drive/My Drive/AI Project/Social_test_AraBERT_Tokenized_Labeled_Data.csv", sep=",")
df_all = pd.read_csv("/content/drive/My Drive/AI Project/All_test_AraBERT_Tokenized_Labeled_Data.csv", sep=",")
df_ASAII = pd.read_csv("/content/drive/My Drive/AI Project/ASAAI10%_Tokenized.csv", sep=",")

In [ ]:
df_sport.head()

,input_ids,attention_mask,Polarity
0,"[2, 34786, 1408, 459, 394, 3150, 2738, 418, 86...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[2, 46395, 50637, 417, 4904, 606, 22034, 11573...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[2, 3270, 232, 357, 545, 6191, 195, 52133, 104...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,"[2, 29920, 394, 305, 1132, 752, 2403, 2738, 31...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,"[2, 29920, 50808, 1132, 195, 624, 6017, 185, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [ ]:
df_sport['input_ids'] = df_sport['input_ids'].apply(ast.literal_eval)
df_sport['attention_mask'] = df_sport['attention_mask'].apply(ast.literal_eval)
test_dataset = MyDataset(df_sport)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)
test('SPORT')

Testing:   0%|          | 0/42 [00:00<?, ?it/s]

REPORT FOR SPORT
              precision    recall  f1-score   support

    NEGATIVE       0.86      0.89      0.87      1649
    POSITIVE       0.88      0.93      0.91      2416
     NEUTRAL       0.81      0.69      0.75      1201

    accuracy                           0.86      5266
   macro avg       0.85      0.84      0.84      5266
weighted avg       0.86      0.86      0.86      5266

0.8619445499430307


0.8619445499430307

In [ ]:
df_tech.head()

,input_ids,attention_mask,Polarity
0,"[2, 44713, 195, 600, 3867, 10818, 26874, 310, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
1,"[2, 39702, 195, 21702, 306, 1416, 30897, 195, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",0
2,"[2, 26546, 3340, 52879, 195, 14740, 22661, 416...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...",0
3,"[2, 8879, 25750, 6085, 759, 3340, 27685, 3, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",0
4,"[2, 8879, 306, 1313, 24591, 195, 5612, 1004, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0


In [ ]:
df_tech['input_ids'] = df_tech['input_ids'].apply(ast.literal_eval)
df_tech['attention_mask'] = df_tech['attention_mask'].apply(ast.literal_eval)
test_dataset = MyDataset(df_tech)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)
test('TECH')

Testing:   0%|          | 0/8 [00:00<?, ?it/s]

REPORT FOR TECH
              precision    recall  f1-score   support

    NEGATIVE       0.94      0.95      0.94       677
    POSITIVE       0.70      0.80      0.74        84
     NEUTRAL       0.89      0.82      0.86       237

    accuracy                           0.91       998
   macro avg       0.84      0.86      0.85       998
weighted avg       0.91      0.91      0.91       998

0.905811623246493


0.905811623246493

In [ ]:
df_political.head()

,input_ids,attention_mask,Polarity
0,"[2, 8527, 11356, 195, 6897, 3945, 20368, 418, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[2, 32260, 5695, 634, 11773, 14179, 1450, 4705...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[2, 1158, 20368, 407, 4346, 21097, 398, 37286,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,"[2, 1158, 18707, 22727, 20368, 407, 11505, 897...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,"[2, 7499, 2209, 3551, 195, 17561, 2409, 306, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [ ]:
df_political['input_ids'] = df_political['input_ids'].apply(ast.literal_eval)
df_political['attention_mask'] = df_political['attention_mask'].apply(ast.literal_eval)
test_dataset = MyDataset(df_political)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)
test('POLITICAL')

Testing:   0%|          | 0/15 [00:00<?, ?it/s]

REPORT FOR POLITICAL
              precision    recall  f1-score   support

    NEGATIVE       0.85      0.87      0.86       757
    POSITIVE       0.86      0.88      0.87       349
     NEUTRAL       0.87      0.83      0.85       791

    accuracy                           0.86      1897
   macro avg       0.86      0.86      0.86      1897
weighted avg       0.86      0.86      0.86      1897

0.8581971534001054


0.8581971534001054

In [ ]:
df_social.head()

,input_ids,attention_mask,Polarity
0,"[2, 1955, 7241, 36795, 9209, 193, 1787, 6023, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[2, 41960, 421, 3917, 9263, 305, 1731, 487, 36...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[2, 6768, 10606, 8867, 21992, 739, 487, 11722,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,"[2, 39290, 14141, 193, 1955, 1601, 195, 57380,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,"[2, 897, 8808, 3346, 740, 193, 25074, 195, 252...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [ ]:
df_social['input_ids'] = df_social['input_ids'].apply(ast.literal_eval)
df_social['attention_mask'] = df_social['attention_mask'].apply(ast.literal_eval)
test_dataset = MyDataset(df_social)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)
test('social')

Testing:   0%|          | 0/25 [00:00<?, ?it/s]

REPORT FOR POLITICAL
              precision    recall  f1-score   support

    NEGATIVE       0.79      0.78      0.79      1044
    POSITIVE       0.75      0.77      0.76       579
     NEUTRAL       0.82      0.82      0.82      1491

    accuracy                           0.80      3114
   macro avg       0.79      0.79      0.79      3114
weighted avg       0.80      0.80      0.80      3114

0.7999357739242132


0.7999357739242132

In [ ]:
df_all.head()

,input_ids,attention_mask,Polarity
0,"[2, 8527, 11356, 195, 6897, 3945, 20368, 418, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[2, 32260, 5695, 634, 11773, 14179, 1450, 4705...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[2, 1158, 20368, 407, 4346, 21097, 398, 37286,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,"[2, 1158, 18707, 22727, 20368, 407, 11505, 897...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,"[2, 7499, 2209, 3551, 195, 17561, 2409, 306, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


In [ ]:
df_all['input_ids'] = df_all['input_ids'].apply(ast.literal_eval)
df_all['attention_mask'] = df_all['attention_mask'].apply(ast.literal_eval)
test_dataset = MyDataset(df_all)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=12, pin_memory=True)
test('ALL')

Testing:   0%|          | 0/89 [00:00<?, ?it/s]

REPORT FOR ALL
              precision    recall  f1-score   support

    NEGATIVE       0.85      0.87      0.86      4127
    POSITIVE       0.86      0.89      0.87      3428
     NEUTRAL       0.83      0.78      0.81      3720

    accuracy                           0.85     11275
   macro avg       0.85      0.85      0.85     11275
weighted avg       0.85      0.85      0.85     11275

0.848070953436807


0.848070953436807

In [ ]:
df_ASAII['input_ids'] = df_ASAII['input_ids'].apply(ast.literal_eval)
df_ASAII['attention_mask'] = df_ASAII['attention_mask'].apply(ast.literal_eval)
test_dataset = MyDataset(df_ASAII)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=12, pin_memory=True)
test('ASAII')

Testing:   0%|          | 0/3 [00:00<?, ?it/s]

REPORT FOR ASAII
              precision    recall  f1-score   support

    NEGATIVE       0.89      0.48      0.63        64
    POSITIVE       0.94      0.29      0.44       118
     NEUTRAL       0.41      0.95      0.58        85

    accuracy                           0.55       267
   macro avg       0.75      0.58      0.55       267
weighted avg       0.76      0.55      0.53       267

0.5468164794007491


0.5468164794007491